In [111]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from osgeo import osr, ogr
import os
from geopandas import gpd
from shapely.geometry import Polygon
from shapely.geometry import Point
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
from sklearn.preprocessing import MinMaxScaler
##Model Classification
from sklearn import tree
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.neighbors import KNeighborsClassifier
os.environ['GDAL_DATA'] ='D:\\Travail\\application\\Anaconda3\\pkgs\\libgdal-2.3.3-h10f50ba_0\\Library\\share\\gdal'
path = 'C:\\Users\\yan_e\\OneDrive\\Bureau\\Hackathon\\pickles\\'

In [2]:
df = pd.read_pickle('beijinglab-china-park.pk')

In [3]:
df = df.loc[df['Name_City'] == '上海']

In [217]:
dfTaxi =pd.read_pickle('utseus-shanghai-taxi-speed-position-GDC2000.pk')


In [6]:
dfTarget = pd.read_pickle('TARGET-communities-extract-wellbeing.pkl')

In [65]:
header = dfTarget[0]
dfTargetF = pd.DataFrame(data=dfTarget, columns=header)
dfTargetF = dfTargetF.drop(dfTargetF.index[0])


,,clean,smell,noise,town,district,longitude,latitude
1,万寿社区居委会,3,0.0,0.0,盈浦街道,青浦区,121.104287016932,31.1558823799679
2,万泰花园第一居委会,3,1.0,0.0,七宝镇,闵行区,121.351529089795,31.144326843509
3,万科阳光苑居委会,2,0.0,0.0,吴泾镇,闵行区,121.468520387505,31.0422853979528
4,万豪居委会,3,0.0,0.0,长征镇,普陀区,121.367237458941,31.2407251034807
5,万里名轩社区居委会,2,,0.0,万里街道,普陀区,121.403601417271,31.269155049366


In [66]:
dfTargetF = dfTargetF[dfTargetF['clean']!='']
dfTargetF = dfTargetF[dfTargetF['smell']!='']
dfTargetF = dfTargetF[dfTargetF['noise']!='']
dfTargetF = dfTargetF[dfTargetF['longitude']!='']
dfTargetF = dfTargetF[dfTargetF['latitude']!='']

dfTargetF['clean'] = dfTargetF['clean'].apply(lambda x: float(x))
dfTargetF['smell'] = dfTargetF['smell'].apply(lambda x: float(x))
dfTargetF['noise'] = dfTargetF['noise'].apply(lambda x: float(x))
dfTargetF['longitude'] = dfTargetF['longitude'].apply(lambda x: float(x))
dfTargetF['latitude'] = dfTargetF['latitude'].apply(lambda x: float(x))

,,clean,smell,noise,town,district,longitude,latitude
1,万寿社区居委会,3.0,0.0,0.0,盈浦街道,青浦区,121.104287,31.155882
2,万泰花园第一居委会,3.0,1.0,0.0,七宝镇,闵行区,121.351529,31.144327
3,万科阳光苑居委会,2.0,0.0,0.0,吴泾镇,闵行区,121.468520,31.042285
4,万豪居委会,3.0,0.0,0.0,长征镇,普陀区,121.367237,31.240725
6,万馨居委会,3.0,0.0,0.0,川沙新镇,浦东新区,121.668506,31.195306


In [219]:
dfMobike = pd.read_pickle('utseus-mobike-2016.pk')

<bound method NDFrame.head of         end_location_x end_location_y  \
0              121.486         31.273   
1              121.455         31.315   
2              121.419         31.156   
3              121.444          31.31   
4               121.46         31.315   
5              121.362         31.283   
6              121.455          31.19   
7              121.423         31.268   
8              121.518         31.291   
9              121.453         31.311   
10             121.405         31.312   
11             121.369          31.18   
12             121.485         31.199   
13             121.479         31.285   
14              121.48         31.275   
15             121.489         31.212   
16             121.315         31.135   
17             121.405         31.294   
18             121.427         31.241   
19             121.471         31.292   
20             121.351         31.312   
21             121.544         31.293   
22             121.506     

<bound method NDFrame.head of          orderid  bikeid userid        start_time start_location_x  \
0              1  324731   7541  2016-08-28 00:00           121.49   
1              6  288841   4427  2016-08-28 00:00          121.462   
2              8  315873   6474  2016-08-28 00:00          121.416   
3             10   93155  12616  2016-08-28 00:00          121.436   
4             11  352484  11596  2016-08-28 00:00          121.459   
5             13  127840   9570  2016-08-28 00:00          121.353   
6             15  125990  15692  2016-08-28 00:00          121.441   
7             16  346549   4373  2016-08-28 00:00          121.426   
8             17  352552    350  2016-08-28 00:00          121.524   
9             20  351480  16317  2016-08-28 00:00          121.442   
10            21  130705  13646  2016-08-28 00:00          121.381   
11            22  348492    840  2016-08-28 00:00          121.407   
12            23  105800  15643  2016-08-28 00:00          1

,,clean,smell,noise,town,district,longitude,latitude
1,万寿社区居委会,3.0,0.0,0.0,盈浦街道,青浦区,121.104287,31.155882
2,万泰花园第一居委会,3.0,1.0,0.0,七宝镇,闵行区,121.351529,31.144327
3,万科阳光苑居委会,2.0,0.0,0.0,吴泾镇,闵行区,121.468520,31.042285
4,万豪居委会,3.0,0.0,0.0,长征镇,普陀区,121.367237,31.240725
6,万馨居委会,3.0,0.0,0.0,川沙新镇,浦东新区,121.668506,31.195306


In [168]:
dfTaxi

,taxi_id,time,status,lon,lat,speed,geometry


In [218]:
dfTaxi= dfTaxi[(dfTaxi['lon']>121.10157)&(dfTaxi['lon']<121.804611)&(dfTaxi['lat']>30.975)&(dfTaxi['lat']<31.514955)]
dfTaxi = dfTaxi.sample(n=10000)

In [220]:
dfMobike = dfMobike.sample(n=10000)

In [131]:
##Fonction to transform longitude and Latitude to metrics position

old_crs = osr.SpatialReference()
old_crs.ImportFromEPSG(4326)
new_crs = osr.SpatialReference()
new_crs.ImportFromEPSG(4479)
transform = osr.CoordinateTransformation(old_crs,new_crs)

def getNewLongitude(xy):
    point = ogr.Geometry(ogr.wkbPoint)
    point.AddPoint(float(xy[0]), float(xy[1]))
    point.Transform(transform)
    return point.GetX()

def getNewLatitude(xy):
    point = ogr.Geometry(ogr.wkbPoint)
    point.AddPoint(float(xy[0]), float(xy[1]))
    point.Transform(transform)
    return point.GetY()

In [221]:
dfTaxi['lon'] = dfTaxi[['lon', 'lat']].apply(getNewLongitude, axis=1)
dfTaxi['lat'] = dfTaxi[['lon', 'lat']].apply(getNewLatitude, axis=1)


In [100]:
dfTargetF['longitude'] = dfTargetF[['longitude', 'latitude']].apply(getNewLongitude, axis=1)
dfTargetF['latitude'] = dfTargetF[['longitude', 'latitude']].apply(getNewLatitude, axis=1)


In [119]:
df.geometry= df.geometry.centroid

In [222]:
dfMobike['end_location_x'] = dfMobike[['end_location_x', 'end_location_y']].apply(getNewLongitude, axis=1)
dfMobike['end_location_y'] = dfMobike[['end_location_x', 'end_location_y']].apply(getNewLatitude, axis=1)


In [172]:
dfTaxi.head

<bound method NDFrame.head of          taxi_id        time  status           lon           lat  speed  \
1471045    18047  2015-04-11       1 -2.852097e+06 -2.700190e+05   38.3   
1328897    15977  2015-04-11       1 -2.845871e+06 -5.164151e+06    0.0   
765161     13604  2015-04-11       0 -2.848501e+06  7.578233e+04   57.2   
541090     21519  2015-04-11       1 -2.847524e+06  5.286882e+06    0.0   
982727     28628  2015-04-11       0 -2.853835e+06 -4.940241e+06   47.2   
109211       127  2015-04-11       0 -2.850602e+06 -4.640170e+06    0.0   
568799     15798  2015-04-11       0 -2.845774e+06  2.364263e+06    0.0   
1177015    10409  2015-04-11       0 -2.842454e+06  5.237960e+06    0.0   
1477813    17274  2015-04-11       0 -2.841227e+06 -5.219694e+06   60.7   
593103     18846  2015-04-11       0 -2.852291e+06 -1.007756e+06    0.0   
577147     16174  2015-04-11       0 -2.839287e+06  2.957572e+06    0.0   
1274190    11330  2015-04-11       1 -2.841926e+06 -5.444875e+06    0.

In [142]:
gdf.set_geometry('geometry')
gdf.crs="+proj=geocent +ellps=GRS80 +units=m +no_defs"
gdf.head()

RuntimeError: b'geocentric transformation missing z or ellps'

In [223]:
nbMob=0
nbTax=0
nbPark=0
k=0
X=[]
Y=[]
for jndex , tab in dfTargetF.iterrows():
    lonUse=tab['longitude']
    latUse=tab['latitude']
    for index , adddfMobike in dfMobike.iterrows():
        if ((lonUse<adddfMobike['end_location_x']+500)& (adddfMobike['end_location_x']-500<lonUse) & (latUse<500+adddfMobike['end_location_y'] )& (adddfMobike['end_location_y']-500<latUse)):
            nbMob=nbMob+1
    for index , adddfTaxi in dfTaxi.iterrows():
         if ((lonUse<adddfTaxi['lon']+500)& (adddfTaxi['lon']-500<lonUse) & (latUse<500+adddfTaxi['lat'] )& (adddfTaxi['lat']-500<latUse)):
            nbTax=nbTax+1
    for index, adddfPark in df.iterrows():
          if ((lonUse<adddfPark['geometry'].x+500)& (adddfPark['geometry'].x-500<lonUse) & (latUse<500+adddfPark['geometry'].y )& (adddfPark['geometry'].y-500<latUse)):
            nbPark=nbPark+1
    X.append([tab['longitude'],tab['latitude'],nbMob,nbTax,nbPark])
    Y.append(tab['noise'])
    print(k)
    k=k+1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [213]:
X

[[-2822207.5865446716, -1174367.0481933358, 0, 0, 0],
 [-2842710.938459042, -2654001.27739947, 0, 0, 0],
 [-2855282.42476444, -4616847.826776564, 0, 0, 0],
 [-2841108.7489435, 1064940.1221043614, 0, 0, 0],
 [-2866944.9660300366, 5439743.876355659, 0, 0, 0],
 [-2844346.8193341065, 1243550.8919890176, 0, 0, 0],
 [-2844477.73407377, -4827253.4686712, 0, 0, 0],
 [-2848874.5420248546, 1369613.2898742978, 0, 1, 0],
 [-2845037.218029534, 3703873.71841932, 0, 1, 0],
 [-2853325.810426851, 3069036.7231907975, 0, 1, 0],
 [-2854366.3121185945, 5233048.055862363, 0, 1, 0]]

In [224]:
## Scale the input data
X = MinMaxScaler().fit_transform(X)

In [225]:
## Creating sample with for train and test datasets
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25,random_state=42)

In [226]:
## Testing the Model
clf = KNeighborsClassifier(weights='distance')
scores = cross_val_score(clf, X_train, Y_train, cv=ShuffleSplit(n_splits=20))
np.mean(scores)

0.7816666666666668

In [227]:
## Predicting the Data test and asses accuracy
clf = KNeighborsClassifier(weights='distance')
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)
np.mean(Y_pred == Y_test)

0.7346938775510204